In [ ]:
!pip install keras==2.15.0
#Import Os and Basis Libraries
import cv2
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
#Matplot Images
import matplotlib.image as mpimg
# Tensflor and Keras Layer and Model and Optimize and Loss
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
# import tensorflow_hub as hub
import optuna
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.applications import ResNet50, InceptionV3, Xception, VGG16
from tensorflow.keras.layers import Dense
import numpy as np
#Image Generator DataAugmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
#Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
# Warnings Remove 
import warnings 
warnings.filterwarnings("ignore")


In [ ]:
# Directory containing the "Train" folder
directory = "/kaggle/input/whistles-baby-hit-me-up/media/DOLPHIN_ALEXIS/Analyses_alexis/dataset/_last/tests"

# List of categories (subfolder names)
categories = ["negatives", "positives"]

# Initialize lists to store filenames and categories
filenames = []
category_labels = []

# Iterate through the categories
for category in categories:
    # Path to the current category folder
    category_folder = os.path.join(directory, "train", category)
    # List all filenames in the category folder
    category_filenames = os.listdir(category_folder)
    # Append filenames and corresponding category labels
    filenames.extend(category_filenames)
    category_labels.extend([category] * len(category_filenames))

# Create DataFrame
df = pd.DataFrame({
    'filename': filenames,
    'category': category_labels
})

# Display the first few rows of the DataFrame
print(df.head())

In [ ]:
#Data_Dir
data_dir = '/kaggle/input/whistles-baby-hit-me-up/media/DOLPHIN_ALEXIS/Analyses_alexis/dataset/_last/tests/train/'

# Defining data generator with Data Augmentation
data_gen_augmented = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

print('Augmented training Images:')
train_ds = data_gen_augmented.flow_from_directory(data_dir, 
                                                              target_size = (224, 224), 
                                                              batch_size = 32,
                                                              subset = 'training',
                                                              class_mode = 'categorical')

#Testing Augmented Data
# Defining Validation_generator withour Data Augmentation
data_gen = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

print('Unchanged Validation Images:')
validation_ds = data_gen.flow_from_directory(data_dir, 
                                        target_size = (224, 224), 
                                        batch_size = 32,
                                        subset = 'validation',
                                        class_mode = 'categorical')

#Testing Augmented Data
test_dir_path = "/kaggle/input/whistles-baby-hit-me-up/media/DOLPHIN_ALEXIS/Analyses_alexis/dataset/_last/tests/test"
# Defining Validation_generator withour Data Augmentation
data_test_gen = ImageDataGenerator(rescale = 1/255.)

print('Test Validation Images:')
test_ds = data_gen.flow_from_directory(test_dir_path, 
                                        target_size = (224, 224), 
                                        batch_size = 32,
                                        subset = 'validation',
                                        class_mode = 'categorical')

In [ ]:
#model_paper = tf.keras.models.load_model("/kaggle/input/cnn_v1_xception/tensorflow2/initial_model/1/model_vgg.h5") #tf & keras ==2.15.0

#Early_Stopping
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, 
    restore_best_weights=True,
)

#Fitting Model
"""history = model_paper.fit(train_ds,
                        epochs= 15,
                        steps_per_epoch = len(train_ds),
                        validation_data = validation_ds,
                        validation_steps = len(validation_ds),
                        callbacks = early_stopping)"""

In [1]:
#model_paper.save("kaggle/working/model_finetuned_vgg.h5")
model_paper= tf.keras.models.load_model("/kaggle/input/cnn_v1_xception/tensorflow2/model_initial_finetuned_95acc/1/model_finetuned_vgg.h5")

NameError: name 'tf' is not defined

In [ ]:
import sys
sys.path.append('/kaggle/input/utility_script')
from utility_script import *


#********************* MAIN
model_path = "/kaggle/input/cnn_v1_xception/tensorflow2/initial_model/1/model_vgg.h5"
images_path = "/kaggle/input/whistles-baby-hit-me-up/media/DOLPHIN_ALEXIS/Analyses_alexis/dataset/_last/tests/train/negatives"
positive_dir = "/kaggle/working/tests/train/negatives/train/negatives/positive"
negative_dir = "/kaggle/working/tests/train/negatives/train/negatives/negative"
if not os.path.exists(positive_dir):
    os.makedirs(positive_dir)
if not os.path.exists(negative_dir):
    os.makedirs(negative_dir)
csv_path = "/kaggle/working/tests/whistles.csv"

# the model
model = tf.keras.models.load_model(model_path)

# paths of all files
all_files_path = [f for f in os.listdir(images_path) if isfile(join(images_path, f))]

# lists to store data
record_names = []
positive_initial = []
positive_finish = []
class_1_scores = []

# all predictions results
predictions = []

# reading file paths 1 by 1
for file_path in all_files_path:
    
    # prediction on the given image
    prediction = predict(model, images_path, file_path)
    prediction_1= predict(model_paper, images_path, file_path)
    predictions.append([file_path, prediction, prediction_1])
    
    # if the class 1 has higher confidence than class 0
    if (prediction[0][1] > prediction[0][0]):
        
        # carry the positive image to its folder.
        copy_file(images_path, file_path, positive_dir)
        
        # storing the positive images confidences
        class_1_scores.append((prediction[0][1],prediction_1[0][1]))
        
        # preparing arrays for the csv
        record_names, positive_initial, positive_finish = prepare_csv_data(file_path,
                                                                           record_names,
                                                                           positive_initial,
                                                                           positive_finish)
    else:
        pass
        # carry the negative image to its folder.
        #copy_file(images_path, file_path, negative_dir)
        
    

#saving the csv
save_csv(record_names, positive_initial, positive_finish, class_1_scores, csv_path)